In [3]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [4]:
#Importing relevant libraries
import requests
import pandas as pd
import numpy as np
import folium
import lxml
import geocoder
from sklearn.cluster import KMeans
print("Libraries imported :)")

Libraries imported :)


In [5]:
#Extracting data from Wikipedia
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
url_data = requests.get(url)

In [6]:
#Cleaning data
tor = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
tor

,Postal Code,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,NaN,NaN
176,M6Z,NaN,NaN
177,M7Z,NaN,NaN
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [7]:
#Cleaning data
tor.dropna(subset=['Borough'], inplace=True)
tor['Neighbourhood'].fillna(tor['Borough'], inplace=True)
tor.reset_index(inplace=True)
tor

,index,Postal Code,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...,...
98,160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,165,M4Y,Downtown Toronto,Church and Wellesley
100,168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [8]:
#Saving to file
tor.to_csv('tor_capstone.csv', index=False)

In [9]:
#Extracting the geographical coordinates
def get_latlng(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
#Checking the format for the last code in the set
get_latlng('M8Z')

[43.62513000000007, -79.52680999999995]

In [11]:
#Renaming one column to remove space
tor2 = tor.rename(columns = {'Postal Code': 'Postalcode'}, inplace = False)
tor2

,index,Postalcode,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...,...
98,160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,165,M4Y,Downtown Toronto,Church and Wellesley
100,168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [12]:
#Creating the second set with geographical coordinates
postal_codes = tor2['Postalcode']    
coords = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]

In [13]:
#Merging two lists into one dataframe
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
tor2['Latitude'] = df_coords['Latitude']
tor2['Longitude'] = df_coords['Longitude']

In [14]:
#Checking the results
tor2

,index,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.75245,-79.32991
1,3,M4A,North York,Victoria Village,43.73057,-79.31306
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
...,...,...,...,...,...,...
98,160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65319,-79.51113
99,165,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
100,168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.64869,-79.38544
101,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.63278,-79.48945


In [15]:
#Saving to file
tor2.to_csv('tor_capstone2.csv', index=False)

In [16]:
#Sorting and grouping - checking the names
tor2.groupby('Borough').count()['Neighbourhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighbourhood, dtype: int64

In [17]:
#Checking the dimensions
len(tor2['Borough'].unique()),tor2.shape[0]

(10, 103)

In [18]:
#Filtering to include Toronto only
tor_map = tor2[tor2['Borough'].str.contains('Toronto')].reset_index(drop=True)
tor_map.head(10)

,index,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
2,13,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
3,22,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
4,30,M4E,East Toronto,The Beaches,43.67709,-79.29547
5,31,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306
6,40,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493
7,41,M6G,Downtown Toronto,Christie,43.66869,-79.42071
8,49,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.64970,-79.38258
9,50,M6H,West Toronto,"Dufferin, Dovercourt Village",43.66505,-79.43891


In [19]:
#Sorting and grouping - checking completeness
tor_map.groupby('Borough').count()['Neighbourhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighbourhood, dtype: int64

In [20]:
#Determining Toronto geographical coordinates
tor_lat = tor_map['Latitude'].mean()
tor_lon = tor_map['Longitude'].mean()
print(tor_lat)
print(tor_lon)

43.66725871794876
-79.39103769230766


In [21]:
#Creating a map of Toronto
toronto_map = folium.Map(location=[tor_lat, tor_lon], zoom_start=12)
toronto_map

In [22]:
#Adding Foursquare information
CLIENT_ID = 'EBTNLXDSN4V13BS4G35IK5WK51WEP5I1NYA0UNWSSJ4DLTTR'
CLIENT_SECRET = 'TUVKCR2Q2S1I5YWVLPYKBYN5JU20NL4RVDLYP0ZDXKRRS000'
VERSION = '20201212'
LIMIT = 500
radius = 500

In [23]:
#Extracting the venues data from Foursquare
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        #Requesting data
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        #Extracting the data    
        results = requests.get(url).json()["response"]['groups'][0]['items']
        #Requesting the data that we need
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
   #Creating the dataframe
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
#Extracting the venues names
toronto_venues = getNearbyVenues(names=tor_map['Neighbourhood'],
                                latitudes=tor_map['Latitude'],
                                longitudes=tor_map['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [25]:
#Sorting and grouping -  checking completeness
toronto_venues.groupby('Neighborhood').count().head(10)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,65,65,65,65,65,65
"Brockton, Parkdale Village, Exhibition Place",84,84,84,84,84,84
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",78,78,78,78,78,78
Central Bay Street,62,62,62,62,62,62
Christie,11,11,11,11,11,11
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,27,27,27,27,27,27


In [26]:
#Extracting the unique names of venues category to get the specific one for gym/fitness venues 
toronto_venues['Venue Category'].unique()

array(['Bakery', 'Coffee Shop', 'Breakfast Spot', 'Greek Restaurant',
       'Yoga Studio', 'Spa', 'Pub', 'Italian Restaurant', 'Event Space',
       'Restaurant', 'Thai Restaurant', 'Distribution Center', 'Theater',
       'Gym / Fitness Center', 'Food Truck', 'Electronics Store',
       'Thrift / Vintage Store', 'Health Food Store', 'Wine Shop', 'Park',
       'Fried Chicken Joint', 'Gastropub', 'Burrito Place', 'Bank',
       'Sandwich Place', 'Café', 'Falafel Restaurant',
       'Mediterranean Restaurant', 'Clothing Store', 'Pizza Place',
       'Plaza', 'Music Venue', 'Comic Shop', 'Ramen Restaurant', 'Hotel',
       'Burger Joint', 'Art Gallery', 'Diner', 'College Rec Center',
       'Japanese Restaurant', 'Steakhouse', 'Middle Eastern Restaurant',
       'Mexican Restaurant', 'Lounge', 'Tanning Salon',
       'Fast Food Restaurant', 'Sporting Goods Shop', 'Shopping Mall',
       'Lake', 'Bookstore', 'Cosmetics Shop',
       'Modern European Restaurant', 'Movie Theater',
       '

In [27]:
# Adding the neighborhood column to the data frame and moving it to the front
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
to_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]
print(to_onehot.shape)
to_onehot.head()

(1713, 227)


,Neighborhoods,Accessories Store,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [28]:
# Sorting and grouping by neighborhoods and means per each venue category
tor_gyms = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()
tor_gyms

,Neighborhoods,Accessories Store,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.015385,0.000000,0.015385,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.015385,0.000000,0.000000,0.000000,0.000000,0.000000,0.015385
1,"Brockton, Parkdale Village, Exhibition Place",0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,...,0.000000,0.00,0.000000,0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905
2,"Business reply mail Processing Centre, South C...",0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.020000,0.000000,...,0.000000,0.00,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.016129,0.016129,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.016129,0.016129,0.016129,0.000000,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658
7,"Commerce Court, Victoria Hotel",0.000000,0.040000,0.000000,0.000000,0.010000,0.000000,0.000000,0.020000,0.000000,...,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.037037,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [29]:
#Checking the number of neighborhooods with gym/fitness venues
len(tor_gyms[tor_gyms["Gym / Fitness Center"] > 0])

12

In [30]:
#Creating a list containing neighoods and a mean of gym/fitness venues
tor_gym = tor_gyms[["Neighborhoods","Gym / Fitness Center"]]
tor_gym.head(10)

,Neighborhoods,Gym / Fitness Center
0,Berczy Park,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000
2,"Business reply mail Processing Centre, South C...",0.010000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.025641
4,Central Bay Street,0.016129
5,Christie,0.000000
6,Church and Wellesley,0.012658
7,"Commerce Court, Victoria Hotel",0.010000
8,Davisville,0.000000
9,Davisville North,0.125000


In [31]:
#Clustering neighborhoods and checking if there's only 3 clusters, as requested
from sklearn.cluster import KMeans
toclusters = 3
tor_clusters = tor_gym.drop(["Neighborhoods"], 1)
kmeans = KMeans(n_clusters=toclusters, random_state=1)
kmeans.fit_transform(tor_clusters)
kmeans.labels_[0:20]

array([2, 2, 2, 0, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [32]:
#Adding cluster labes to the list of neighborhoods and means of gym/fitness venues
tor_gym2 = tor_gym.copy()
tor_gym2["Cluster Labels"] = kmeans.labels_

In [33]:
#Remaning one column for merging purposes
tor_gym2.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
tor_gym2.head(10)

,Neighborhood,Gym / Fitness Center,Cluster Labels
0,Berczy Park,0.000000,2
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,2
2,"Business reply mail Processing Centre, South C...",0.010000,2
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.025641,0
4,Central Bay Street,0.016129,0
5,Christie,0.000000,2
6,Church and Wellesley,0.012658,2
7,"Commerce Court, Victoria Hotel",0.010000,2
8,Davisville,0.000000,2
9,Davisville North,0.125000,1


In [34]:
#Merging two lists
tor_gym2 = tor_gym2.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")
tor_gym2

,Neighborhood,Gym / Fitness Center,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.0,2,43.64536,-79.37306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.0,2,43.64536,-79.37306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.0,2,43.64536,-79.37306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.0,2,43.64536,-79.37306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.0,2,43.64536,-79.37306,St. Lawrence Market (South Building),43.648743,-79.371597,Farmers Market
...,...,...,...,...,...,...,...,...,...
38,"University of Toronto, Harbord",0.0,2,43.66311,-79.40180,Second Cup (Miles Nadal JCC Fitness),43.666527,-79.403872,Coffee Shop
38,"University of Toronto, Harbord",0.0,2,43.66311,-79.40180,Shoppers Drug Mart,43.666562,-79.405007,Pharmacy
38,"University of Toronto, Harbord",0.0,2,43.66311,-79.40180,Al Green Theatre,43.666547,-79.404053,Concert Hall
38,"University of Toronto, Harbord",0.0,2,43.66311,-79.40180,Kenzo Japanese Noodle House,43.666295,-79.406002,Ramen Restaurant


In [35]:
#Creating a list of cluster 0 venues
tor_gym2.loc[(tor_gym2['Cluster Labels'] ==0) & (tor_gym2['Venue Category'] == 'Gym / Fitness Center') ]

,Neighborhood,Gym / Fitness Center,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.025641,0,43.64082,-79.39818,FlirtyGirl Fitness,43.644005,-79.397590,Gym / Fitness Center
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.025641,0,43.64082,-79.39818,Cykl,43.642778,-79.402361,Gym / Fitness Center
4,Central Bay Street,0.016129,0,43.65609,-79.38493,Hard Candy Fitness,43.659556,-79.382440,Gym / Fitness Center
24,"Regent Park, Harbourfront",0.041667,0,43.65512,-79.36264,The Extension Room,43.653313,-79.359725,Gym / Fitness Center


In [36]:
#Creating a list of cluster 1 venues
tor_gym2.loc[(tor_gym2['Cluster Labels'] ==1) & (tor_gym2['Venue Category'] == 'Gym / Fitness Center') ]

,Neighborhood,Gym / Fitness Center,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
9,Davisville North,0.125,1,43.71276,-79.38851,900 Mount Pleasant - Residents Gym,43.711671,-79.391767,Gym / Fitness Center


In [37]:
#Creating a list of cluster 2 venues
tor_gym2.loc[(tor_gym2['Cluster Labels'] ==2) & (tor_gym2['Venue Category'] == 'Gym / Fitness Center') ]

,Neighborhood,Gym / Fitness Center,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2,"Business reply mail Processing Centre, South C...",0.010000,2,43.64869,-79.38544,Adelaide Club Toronto,43.649279,-79.381921,Gym / Fitness Center
6,Church and Wellesley,0.012658,2,43.66659,-79.38133,Verve Gym,43.666702,-79.376539,Gym / Fitness Center
7,"Commerce Court, Victoria Hotel",0.010000,2,43.64840,-79.37914,Adelaide Club Toronto,43.649279,-79.381921,Gym / Fitness Center
11,"First Canadian Place, Underground city",0.010000,2,43.64828,-79.38146,Adelaide Club Toronto,43.649279,-79.381921,Gym / Fitness Center
13,"Garden District, Ryerson",0.010000,2,43.65739,-79.37804,Hard Candy Fitness,43.659556,-79.382440,Gym / Fitness Center
25,"Richmond, Adelaide, King",0.010000,2,43.64970,-79.38258,Adelaide Club Toronto,43.649279,-79.381921,Gym / Fitness Center
31,Stn A PO Boxes,0.010000,2,43.64869,-79.38544,Adelaide Club Toronto,43.649279,-79.381921,Gym / Fitness Center
37,"Toronto Dominion Centre, Design Exchange",0.010000,2,43.64710,-79.38153,Adelaide Club Toronto,43.649279,-79.381921,Gym / Fitness Center


In [38]:
toronto_map2 = folium.Map(location=[tor_lat, tor_lon],zoom_start=12)
markers_colors={}
markers_colors[0] = 'yellow'
markers_colors[1] = 'magenta'
markers_colors[2] = 'cyan'
for lat,lon,cluster in zip(tor_gym2['Neighborhood Latitude'],tor_gym2['Neighborhood Longitude'],tor_gym2['Cluster Labels']):
    folium.features.CircleMarker(
        [lat, lon],
        radius=5,   
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.5).add_to(toronto_map2)

In [39]:
toronto_map2